Michał Zych, michal.zych96@gmail.com

Michał Warzecha, warzmich@gmail.com

In [0]:
%tensorflow_version 1.x
import keras
keras.__version__

TensorFlow 1.x selected.


Using TensorFlow backend.


'2.2.5'

In [0]:
from keras.datasets import reuters

(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

2113536/2110848 [==============================] - 1s 1us/step


In [0]:
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

# Our vectorized training data
x_train = vectorize_sequences(train_data)
# Our vectorized test data
x_test = vectorize_sequences(test_data)

In [0]:
def to_one_hot1(labels, dimension=46):
    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels):
        results[i, label] = 1.
    return results

def to_one_hot(labels, dimension=46):
    results = []
    
    for label in labels:
        results.append(np.asarray([label==i for i in range(0,dimension)],dtype=np.uint8))
        
    return np.asarray(results)
    
# Our vectorized training labels
one_hot_train_labels = to_one_hot(train_labels)
# Our vectorized test labels
one_hot_test_labels = to_one_hot(test_labels)

print(one_hot_train_labels.shape)

(8982, 46)


In [0]:
from keras.utils.np_utils import to_categorical

one_hot_train_labels = to_categorical(train_labels)
one_hot_test_labels = to_categorical(test_labels)

Przygotowanie zbioru walidacyjnego

In [0]:
x_val = x_train[:1000]
partial_x_train = x_train[1000:]

y_val = one_hot_train_labels[:1000]
partial_y_train = one_hot_train_labels[1000:]

Po przygotowaniu danych można rozpacząć trening. 

In [0]:
from keras import models
from keras import layers
from sklearn.metrics import classification_report, confusion_matrix

def create_model(nr_elements, nr_hidden_layers, activation):
  model = models.Sequential()
  model.add(layers.Dense(nr_elements, activation=activation, input_shape=(10000,)))
  for i in range(nr_hidden_layers-1):
    model.add(layers.Dense(nr_elements, activation=activation))
  model.add(layers.Dense(46, activation='softmax'))
  return model 

In [0]:

# TEST FOR DIFFEREN NR ELEMENTS AND NR OF HIDDEN LAYER

print("ALL NETWORKS ON THIS SET HAVE LOSS FUNCTION: CATEGORICAL_CROSSENTROPY AND ACTIVATION: RELU")
print("****************************************\n\n")
for nr_elements in [32, 64, 128]:
  for nr_hidden_layers in [1,2,3]:
    model = create_model(nr_elements, nr_hidden_layers, 'relu')
    model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    print("Nr elements: " + str(nr_elements) + ", nr hidden_layers: " + str(nr_hidden_layers))
    model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=0)          # TOO MANY ITERATIONS AND PRINT TRAINING WOULD BE UNREADABLE
    predicts = model.predict(x_test)
    predicted_labels = np.argmax(predicts, axis=1)
    print(classification_report(test_labels, predicted_labels))
    print(confusion_matrix(test_labels, predicted_labels))
    print("****************************************\n\n")

ALL NETWORKS ON THIS SET HAVE LOSS FUNCTION: CATEGORICAL_CROSSENTROPY AND ACTIVATION: RELU
****************************************


Nr elements: 32, nr hidden_layers: 1


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.86      0.50      0.63        12
           1       0.65      0.81      0.72       105
           2       0.87      0.65      0.74        20
           3       0.90      0.94      0.92       813
           4       0.78      0.90      0.83       474
           5       1.00      0.20      0.33         5
           6       0.92      0.79      0.85        14
           7       1.00      0.33      0.50         3
           8       0.74      0.66      0.69        38
           9       0.86      0.72      0.78        25
          10       0.92      0.77      0.84        30
          11       0.61      0.75      0.67        83
          12       0.40      0.15      0.22        13
          13       0.61      0.62      0.61        37
          14       0.00      0.00      0.00         2
          15       0.50      0.11      0.18         9
          16       0.71      0.76      0.74        99
          17       1.00    

Wszystkie sieci wykazały zbliżony poziom dokładnosci (między 80 a 75 % poprawnej predykcji) co ciekawe najlepszą skuteczność miały sieci jednowarstwowe ewentualnie dwuwarstwowe. Możliwe że dołożenie trzeciej warstwy przy takiej liczbie danych przyczyniło się do delikatnego przetrenowania modelu i gorszej dokładności

In [0]:
#TEST FOR OTHER ACTIVATION (TANH) AND OTHER LOSS FUNCTION (MSE)


print("ACTIVATION: TANH")
model = create_model(64, 2, 'tanh')
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
print("Nr elements: " + str(64) + ", nr hidden_layers: " + str(2))
model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=0)          # TOO MANY ITERATIONS AND PRINT TRAINING WOULD BE UNREADABLE
predicts = model.predict(x_test)
predicted_labels = np.argmax(predicts, axis=1)
print(classification_report(test_labels, predicted_labels))
print(confusion_matrix(test_labels, predicted_labels))


print("****************************************\n\n")
print("LOSS FUNCTION: MSE")
model = create_model(64, 2, 'relu')
model.compile(optimizer='rmsprop',
              loss='mean_squared_error',
              metrics=['accuracy'])
print("Nr elements: " + str(64) + ", nr hidden_layers: " + str(2))
model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=0)          # TOO MANY ITERATIONS AND PRINT TRAINING WOULD BE UNREADABLE
predicts = model.predict(x_test)
predicted_labels = np.argmax(predicts, axis=1)
print(classification_report(test_labels, predicted_labels))
print(confusion_matrix(test_labels, predicted_labels))

ACTIVATION: TANH
Nr elements: 64, nr hidden_layers: 2


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.86      0.50      0.63        12
           1       0.66      0.81      0.73       105
           2       0.73      0.55      0.63        20
           3       0.92      0.91      0.91       813
           4       0.79      0.89      0.84       474
           5       0.50      0.20      0.29         5
           6       0.92      0.86      0.89        14
           7       1.00      0.33      0.50         3
           8       0.70      0.68      0.69        38
           9       0.80      0.80      0.80        25
          10       0.86      0.83      0.85        30
          11       0.61      0.72      0.66        83
          12       0.40      0.15      0.22        13
          13       0.55      0.57      0.56        37
          14       1.00      0.50      0.67         2
          15       0.50      0.11      0.18         9
          16       0.64      0.76      0.69        99
          17       0.60    

Dla pokazanego powyżej przykładu (liczba warstw = 2 liczba elementów w warstwie = 64) zmiana funkcji aktywacji z relu na tanh nie zmieniła dokładności modelu (dalej wynosi ona 78%).

Zmiana loss function z crossentropy_categorical na MSE spowodowała drobne pogorszenie dokładności modelu (spadek z 78 do 77%)